In [56]:
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.layers.experimental import preprocessing

In [250]:
import pathlib
font_csvs =  [str(p) for p in pathlib.Path('preproc_data/afpdb').glob("*.csv")]
train_csvs = font_csvs[0:40]+font_csvs[50:90]
test_csvs = font_csvs[40:50]+font_csvs[90:100]
train_labels = np.array([0]*40*60+[1]*40*60)
test_labels = np.array([0]*10*60+[1]*10*60)
train_labels = train_labels.reshape((-1,1))
test_labels = test_labels.reshape((-1,1))
print("Train labels size: ", train_labels.size)
print("Test label size: ", test_labels.size)

Train labels size:  4800
Test label size:  1200


In [232]:
train_data = np.array([])
for csv in train_csvs:
    data = pd.read_csv(csv, header=None)
    for c in range(data.shape[1]):
        row = np.array([np.array([data.iloc[:, c]])])
        if train_data.shape[0] == 0:
            train_data = row
        else:
            train_data = np.vstack((train_data, row))
train_data = np.moveaxis(train_data, 1, 2)
train_data.shape

(4800, 900, 1)

In [231]:
test_data = np.array([])
for csv in test_csvs:
    data = pd.read_csv(csv, header=None)
    for c in range(data.shape[1]):
        row = np.array([np.array([data.iloc[:, c]])])
        if test_data.shape[0] == 0:
            test_data = row
        else:
            test_data = np.vstack((test_data, row))
test_data = np.moveaxis(test_data, 1, 2)
test_data.shape

(1200, 900, 1)

In [265]:
train = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
test = tf.data.Dataset.from_tensor_slices((test_data, test_labels))

train = train.shuffle(len(train)).batch(2)
test = test.batch(16)

for d, l in train.take(1):
    print ("Data batch size: ", d.shape)
    print ("Labels batch size: ", l.shape)

Data batch size:  (2, 900, 1)
Labels batch size:  (2, 1)


In [267]:
model = models.Sequential()
model.add(layers.BatchNormalization(input_shape=(900,1)))
model.add(layers.Conv1D(16,8))

model.add(layers.MaxPool1D(strides=2))
model.add(layers.Dropout(0.25))
model.add(layers.Conv1D(32,4))
model.add(layers.Conv1D(32,4))
model.add(layers.MaxPool1D(strides=2))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(16, activation='relu'))

model.add(layers.MaxPool1D(strides=2))
model.add(layers.Dropout(0.25))
model.add(layers.Conv1D(32,2))
model.add(layers.Conv1D(32,2))
model.add(layers.MaxPool1D(strides=2))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(8, activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(2, activation='relu'))

model.summary()

Model: "sequential_63"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_61 (Batc (None, 900, 1)            4         
_________________________________________________________________
conv1d_77 (Conv1D)           (None, 893, 16)           144       
_________________________________________________________________
max_pooling1d_57 (MaxPooling (None, 446, 16)           0         
_________________________________________________________________
dropout_56 (Dropout)         (None, 446, 16)           0         
_________________________________________________________________
conv1d_78 (Conv1D)           (None, 443, 32)           2080      
_________________________________________________________________
conv1d_79 (Conv1D)           (None, 440, 32)           4128      
_________________________________________________________________
max_pooling1d_58 (MaxPooling (None, 220, 32)         

In [ ]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.BinaryCrossentropy(),
             metrics=['accuracy'])

history = model.fit(train, epochs=200,
                       validation_data=(test))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

In [54]:
test_loss, test_acc = model.evaluate(test, verbose=2)
print("Test loss: ", test_loss)
Print("Test accuracy: ", test_acc)

Loss:  2
